In [20]:
import os
import gensim
from gensim.models import Word2Vec
import nltk
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob
import pickle
import stanza
from os import path
from unicodedata import normalize
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive

In [11]:
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
# Now you can access the target directory using its path:
path_name = '/content/drive/MyDrive/TAL_ENC/txt/*.txt'

In [62]:
STANZA_PIPE = stanza.Pipeline(lang='fr', processors='tokenize, lemma, pos')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fr (French):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no

In [63]:
def clean_text(txt):
    txt_res = normalize("NFKD", txt).replace('\xa0', ' ')
    txt_res = txt_res.replace("\\", "").replace('\\xa0', '')
    return txt_res

In [67]:
def pipeline_stanza_sentences(doc):

    pos_ko = ["NUM", "X", "SYM", "PUNCT"]
    list_sentences_tokens, list_sentences_lemmas, list_sentences_pos, list_sentences_wt_proper_sw = [], [], [], []
    nb_sentences, nb_tokens = 0, 0

    with open(doc, encoding="utf8") as file:
        text = file.readlines()
        text_clean = clean_text(str(text[0]).lower())
        docs = STANZA_PIPE(text_clean)#text_clean
        nb_sentences += len(docs.sentences)

        for sent in tqdm(docs.sentences):
            list_tokens, list_lemmas, list_pos, list_wt_proper_sw = [], [], [], []
            nb_tokens += len(sent.words)
            for word in sent.words:
                #si le token est bien un mot on récupère son texte, son lemme et son pos
                if word.upos not in pos_ko:
                    list_tokens.append(word.parent.text)
                    list_lemmas.append(word.lemma)
                    list_pos.append(word.upos)
		            # si le token n'est pas un stopwords ou un proper noun on le prend
                if word.upos not in ['DET', 'PRON', 'ADP', 'AUX', 'CONJ', 'SCONJ', 'PART', 'PROPN']+pos_ko:
                    list_wt_proper_sw.append(word.parent.text)
            list_sentences_tokens.append(" ".join(map(str, list_tokens)))
            list_sentences_lemmas.append(" ".join(map(str, list_lemmas)))
            list_sentences_pos.append(" ".join(map(str, list_pos)))
            list_sentences_wt_proper_sw.append(" ".join(map(str, list_wt_proper_sw)))


    return list_sentences_tokens, list_sentences_lemmas, list_sentences_pos, list_sentences_wt_proper_sw, nb_sentences, nb_tokens


In [65]:
def moulinette(path_name):

    nb_total_tokens, nb_total_sentences = 0, 0
    main_list_token, main_list_lemma, main_list_pos, main_list_wt_proper_sw, main_list_index = [], [], [], [], []

    #print("\n\nBEGIN PROCESSING CORPUS-----------")

    for doc in tqdm(glob(path_name)):

        #print("\n\nBEGIN PROCESSING NOVEL-----------")

        doc_name = path.splitext(path.basename(doc))[0]
        date = doc_name.split("_")[0]
        print(doc_name)

        #On recupere le texte des romans sous forme de listes de lemmes et de pos grâce à stanza

        list_token, list_lemma, list_pos, list_wt_proper_sw, nb_sentences, nb_tokens = pipeline_stanza_sentences(doc)

        nb_total_tokens += nb_tokens
        nb_total_sentences += nb_sentences

        main_list_token.append(list_token)
        main_list_lemma.append(list_lemma)
        main_list_pos.append(list_pos)
        main_list_wt_proper_sw.append(list_wt_proper_sw)
        main_list_index.append(doc_name)
        #print("\n\nEND PROCESSING NOVEL-----------")

    print("NB TOKENS FINAL: ", nb_total_tokens, "NB SENTENCES FINAL: ", nb_total_sentences)
    #print("\n\nEND PROCESSING CORPUS-----------")

    return main_list_lemma, main_list_token, main_list_pos, main_list_wt_proper_sw, main_list_index

In [69]:
main_list_lemma, main_list_token, main_list_pos, main_list_wt_proper_sw, main_list_index = moulinette(path_name)

In [ ]:
with open('TOKENS_education.pkl', 'wb') as f1, open('LEMMAS_education.pkl', 'wb') as f2, open('FILTERED_education.pkl', 'wb') as f3:
            pickle.dump(tokens[1513], f1)
            pickle.dump(lemmas[1513], f2)
            pickle.dump(filtered[1513], f3)

In [69]:
with open('TOKENS_vingt.pkl', 'wb') as f1, open('LEMMAS_vingt.pkl', 'wb') as f2, open('FILTERED_vingt.pkl', 'wb') as f3:
            pickle.dump(tokens[2728], f1)
            pickle.dump(lemmas[2728], f2)
            pickle.dump(filtered[2728], f3)